# Generate CMA using tick data service - F#

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *cumulative moving average - CMA* from a tick data service.

### Inputs/outputs
Cumulative moving averages' requests require instrument's identifier, date time intervals and number of periods as per inputs.

CMA is the average of all the data points.

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve tick data from the hosted service. The queried endpoint in this script are:
* *TickCmaService *: to directly retrieve tick cma data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run CMA sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.TickAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Core
open Grpc.Net.Client
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *CMA* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Cumulative moving average request parameters

#### 3.1 Instrument selection

In [ ]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 CMA parameters

In [ ]:
let field = CmaPrice.Trade
let sampling = new Duration ( Seconds = 60L )

#### 3.3 Time period delimitation

In [ ]:
// Build the bars request date interval (we are using Google date time format : we have to cast the dates)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = 2020, Month = 02, Day = 18 ),
    EndDate = new Date ( Year = 2020, Month = 02, Day = 18 )
  )

// Build the bars request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
  new TimeInterval(
    StartTime = new TimeOfDay ( Hours = 08, Minutes = 00, Seconds = 00 ),
    EndTime = new TimeOfDay ( Hours = 20, Minutes = 00, Seconds = 00 ) 
  )

#### 3.4 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
let constraints = new Constraints()
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)   

The following code snippets create requests for: *Long SMA*, *Short SMA* and *Prices*:

In [ ]:
// Generate CMA request
let request = 
    new TickCmaRequest(
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker), 
        Constraints = constraints,  
        Field = field,
        Sampling = sampling
    )

// Instantiate the tick CMA service
let service = new TickCmaService.TickCmaServiceClient(channel)

### Step 4: Visualize data

#### 4.1 Retrieve CMA data

In [ ]:
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()

let cmas = new List<TickCmaResponse>()

let call = service.TickCma(request, headers)
let e = call.ResponseStream.ReadAllAsync()

iterAsync e cmas.Add

#### 4.4 Plot CMA and prices
Plot CMA and prices request' results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. Plot a simple line graph with the CMA data points and prices: 

In [ ]:
// Plot simultaneously prices and cumulative moving averages
[
    new Scattergl(
        name = "CMA",
        mode = "lines",
        x = (cmas |> Seq.map (fun l -> l.TimeStamp.ToDateTime())),
        y = (cmas |> Seq.map (fun l -> l.Average))
    )
    new Scattergl(
        name = "Prices",
        mode = "lines",
        x = (cmas |> Seq.map (fun l -> l.TimeStamp.ToDateTime())),
        y = (cmas |> Seq.map (fun l -> l.Value))
    )
]
|> Chart.Plot
|> Chart.WithTitle $"CMA and prices for {ticker}"
|> Chart.WithXTitle $"Time"
|> Chart.WithYTitle "Price"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display